In [2]:
import requests
import json
import numpy as np
import pandas as pd

DISP_IDS = {
  "flowertopia": "614cdcbdea191200a8a1ed2f",
  "toke" : "5f8758560d683700eab65918",
  "Shiva's rose":"5fdd352ebb29a301034435d5",
  "Growers Retail": "5eb9a5221c1db800f8bd2a7d"
}

COLS = [
    'cName', 
    'name',
    'brandName',
    'Prices',
    'recOnly',
    'recPrices',
    'special', 
    'specialData', 
    'type', 
    'wholesalePrices'
]

def getProducts(id):
  x = requests.get(f'https://dutchie.com/graphql?operationName=FilteredProducts&variables=%7B%22includeEnterpriseSpecials%22%3Afalse%2C%22includeCannabinoids%22%3Atrue%2C%22productsFilter%22%3A%7B%22dispensaryId%22%3A%22{id}%22%2C%22pricingType%22%3A%22rec%22%2C%22strainTypes%22%3A%5B%5D%2C%22subcategories%22%3A%5B%5D%2C%22Status%22%3A%22Active%22%2C%22types%22%3A%5B%5D%2C%22useCache%22%3Afalse%2C%22sortDirection%22%3A1%2C%22sortBy%22%3Anull%2C%22isDefaultSort%22%3Atrue%2C%22bypassOnlineThresholds%22%3Afalse%2C%22isKioskMenu%22%3Afalse%2C%22removeProductsBelowOptionThresholds%22%3Atrue%7D%2C%22page%22%3A0%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22699c79a80fc45daf41c34407247fd0c38dd698fd595d36501b90a3104931ef1a%22%7D%7D').text
  clean = json.loads(x)
  prodList = clean['data']['filteredProducts']['products']
  return prodList

def toDataFrame(prodList):
  infoDataFrame = pd.DataFrame(columns=COLS)
  for prod in prodList:
    row = pd.DataFrame([[prod['cName'], prod['Name'], prod['brandName'], prod['Prices'], prod['recOnly'],prod['recSpecialPrices'], prod['special'], prod['specialData'], prod['type'], prod['wholesalePrices']]],columns=COLS, index=['1'])
    infoDataFrame = infoDataFrame.append(row, ignore_index=True)
  return infoDataFrame


for disp in DISP_IDS:
  id = DISP_IDS[disp]
  prods = getProducts(id)
  df = toDataFrame(prods)
  df.to_csv(path_or_buf=f'./out/{disp}.csv')



NameError: name 'cols' is not defined